# Relay Example Notebook

This notebook demonstrates how to use Relay to:
1. Submit batch jobs
2. Track job progress
3. Retrieve results from old jobs

The key feature of Relay is that all jobs and results are stored in a workspace directory, so you can access them across sessions.

## Installation

First, install the package from PyPI:

```bash
pip install relay-llm
```

Or install from source:

```bash
git clone https://github.com/neelguha/relay.git
cd relay
pip install -e .
```

## Setup

First, let's import the necessary modules and set up the workspace.

**Note:** This notebook assumes you have installed `relay-llm` via pip. If you're running this from the source directory, the imports will work, but for production use, install the package first.

In [1]:
import os
import time

# Import Relay (from pip-installed package)
try:
    from relay import RelayClient, BatchRequest, __version__
    print(f"✓ Relay version {__version__} imported successfully")
except ImportError:
    print("✗ Error: Could not import relay. Make sure you've installed it:")
    print("   pip install relay-llm")
    raise

# Check if API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  Warning: OPENAI_API_KEY not set. Set it with:")
    print("   export OPENAI_API_KEY='your-api-key'")
else:
    print("✓ OPENAI_API_KEY is set")

✓ Relay version 0.1.0 imported successfully
✓ OPENAI_API_KEY is set


## Part 1: Submitting Jobs

Create a workspace and submit some batch jobs.

In [2]:
# Create a workspace - all jobs will be stored here
workspace_dir = "example_workspace"
relay = RelayClient(directory=workspace_dir)

print(f"✓ Created workspace: {workspace_dir}")

✓ Created workspace: example_workspace


In [3]:
# Create some batch requests
requests = [
    BatchRequest(
        id="req-1",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="What is 2+2?",
        provider_args={}
    ),
    BatchRequest(
        id="req-2",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="What is the capital of France?",
        provider_args={}
    ),
    BatchRequest(
        id="req-3",
        model="gpt-4o-mini",
        system_prompt="You are a helpful assistant.",
        prompt="Explain quantum computing in one sentence.",
        provider_args={}
    ),
]

print(f"Created {len(requests)} requests")

Created 3 requests


In [4]:
# Submit the batch job with a unique ID
job_id = "notebook-demo-001"

job = relay.submit_batch(
    requests=requests,
    job_id=job_id,
    provider="openai",
    description="Notebook demonstration batch job"
)

print(f"✓ Job submitted successfully!")
print(f"  Job ID: {job.job_id}")
print(f"  Submitted at: {job.submitted_at}")
print(f"  Status: {job.status}")
print(f"  Number of requests: {job.n_requests}")

✓ Job submitted successfully!
  Job ID: notebook-demo-001
  Submitted at: 2025-12-20 10:44:51.972537
  Status: validating
  Number of requests: 3


If you go to OpenAI's [batch monitoring page](https://platform.openai.com/batches/), you should be able to see this batch.

## Part 2: Tracking Jobs

List all jobs in the workspace and monitor their progress.

In [5]:
# List all jobs in the workspace
all_jobs = relay.list_jobs()
print(f"Found {len(all_jobs)} job(s) in workspace:")
for jid in all_jobs:
    print(f"  - {jid}")

Found 1 job(s) in workspace:
  - notebook-demo-001


In [6]:
# Get detailed information about a specific job
job_info = relay.get_job(job_id)
if job_info:
    print(f"Job: {job_info['job_id']}")
    print(f"  Description: {job_info['description']}")
    print(f"  Provider: {job_info['provider']}")
    print(f"  Status: {job_info['status']}")
    print(f"  Submitted: {job_info['submitted_at']}")
    print(f"  Requests: {job_info['n_requests']}")
    print(f"  Completed: {job_info.get('completed_requests', 0)}")
    print(f"  Failed: {job_info.get('failed_requests', 0)}")

Job: notebook-demo-001
  Description: Notebook demonstration batch job
  Provider: openai
  Status: validating
  Submitted: 2025-12-20T10:44:51.972537
  Requests: 3
  Completed: 0
  Failed: 0


In [7]:
# Monitor job progress
job_status = relay.monitor_batch(job_id)
display(job_status)

BatchJob(job_id='notebook-demo-001', submitted_at=1766245491, status='completed', n_requests=3, completed_requests=3, failed_requests=0)

## Part 3: Getting Results

Next, we can get the results from this job.

In [9]:
results = relay.retrieve_batch_results(job_id)
    
print(f"\n✓ Retrieved {len(results)} results")
print(f"\nResults are automatically saved to: {workspace_dir}/{job_id}_results.json")

# Display sample results
print("\n" + "=" * 60)
print("Sample Results:")
print("=" * 60)

for i, result in enumerate(results[:3], 1):  # Show first 3
    print(f"\nResult {i}:")
    custom_id = result.get('custom_id', 'N/A')
    print(f"  Request ID: {custom_id}")
    
    # Extract response based on OpenAI format
    if 'response' in result:
        response = result['response']
        if 'body' in response:
            body = response['body']
            if 'output' in body:
                output = body['output']
                # Truncate long outputs
                output_str = str(output)
                if len(output_str) > 200:
                    output_str = output_str[:200] + "..."
                print(f"  Output: {output_str}")
    
    if 'error' in result:
        print(f"  Error: {result['error']}")

if len(results) > 3:
    print(f"\n  ... and {len(results) - 3} more results")


✓ Retrieved 3 results

Results are automatically saved to: example_workspace/notebook-demo-001_results.json

Sample Results:

Result 1:
  Request ID: req-1
  Output: [{'id': 'msg_0d27a8a2f2db6732006946c4d876108196b2cb48c5995d3455', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': '2 + 2 equals...
  Error: None

Result 2:
  Request ID: req-2
  Output: [{'id': 'msg_0eadac4221c325ff006946c4d10fac81908d5a6350d95e39db', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': 'The capital ...
  Error: None

Result 3:
  Request ID: req-3
  Output: [{'id': 'msg_0c410940f0c6eb7b006946c4d2283c819096b5f249214f95d0', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': 'Quantum comp...
  Error: None


## Part 4: Getting Results from Old Jobs

We can also get results from old jobs.

In [10]:
new_relay = RelayClient(directory=workspace_dir)
print("✓ New RelayClient created")

# All jobs are still accessible!
existing_jobs = new_relay.list_jobs()
print(f"\nFound {len(existing_jobs)} existing job(s):")
for jid in existing_jobs:
    print(f"  - {jid}")

✓ New RelayClient created

Found 1 existing job(s):
  - notebook-demo-001


In [11]:
# Check if results exist for a job
if new_relay.has_results(job_id):
    print(f"✓ Results exist for {job_id}")
else:
    print(f"✗ No results found for {job_id}")
    print("  (Results are saved when you call retrieve_batch_results)")

✓ Results exist for notebook-demo-001


In [12]:
results = new_relay.retrieve_batch_results(job_id)
    
print(f"\n✓ Retrieved {len(results)} results")
print(f"\nResults are automatically saved to: {workspace_dir}/{job_id}_results.json")

# Display sample results
print("\n" + "=" * 60)
print("Sample Results:")
print("=" * 60)

for i, result in enumerate(results[:3], 1):  # Show first 3
    print(f"\nResult {i}:")
    custom_id = result.get('custom_id', 'N/A')
    print(f"  Request ID: {custom_id}")
    
    # Extract response based on OpenAI format
    if 'response' in result:
        response = result['response']
        if 'body' in response:
            body = response['body']
            if 'output' in body:
                output = body['output']
                # Truncate long outputs
                output_str = str(output)
                if len(output_str) > 200:
                    output_str = output_str[:200] + "..."
                print(f"  Output: {output_str}")
    
    if 'error' in result:
        print(f"  Error: {result['error']}")

if len(results) > 3:
    print(f"\n  ... and {len(results) - 3} more results")


✓ Retrieved 3 results

Results are automatically saved to: example_workspace/notebook-demo-001_results.json

Sample Results:

Result 1:
  Request ID: req-1
  Output: [{'id': 'msg_0d27a8a2f2db6732006946c4d876108196b2cb48c5995d3455', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': '2 + 2 equals...
  Error: None

Result 2:
  Request ID: req-2
  Output: [{'id': 'msg_0eadac4221c325ff006946c4d10fac81908d5a6350d95e39db', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': 'The capital ...
  Error: None

Result 3:
  Request ID: req-3
  Output: [{'id': 'msg_0c410940f0c6eb7b006946c4d2283c819096b5f249214f95d0', 'type': 'message', 'status': 'completed', 'content': [{'type': 'output_text', 'annotations': [], 'logprobs': [], 'text': 'Quantum comp...
  Error: None
